# Compressão de Imagem com K-means

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def find_closest_centroids(X, centroids):
    # Número de centróides (K)
    K = centroids.shape[0]
    
    # Inicializa o array de índices com zeros, do mesmo tamanho que o número de exemplos em X
    idx = np.zeros(X.shape[0], dtype=int)

    # Para cada exemplo em X
    for i in range(X.shape[0]):
        # Calcula a distância entre o exemplo X[i] e todos os centróides
        distances = np.linalg.norm(X[i] - centroids, axis=1)
        
        # Atribui o índice do centróide mais próximo ao exemplo X[i]
        idx[i] = np.argmin(distances)
    
    return idx

def plot_progress_kMeans(X, centroids, previous_centroids, idx, K, i):
   
    # Define as cores para os diferentes clusters
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

    # Plota os exemplos de treinamento
    for k in range(K):
        # Seleciona exemplos pertencentes ao cluster k
        cluster_points = X[idx == k]
        plt.scatter(cluster_points[:, 0], cluster_points[:, 1], s=30, color=colors[k % len(colors)], label=f'Cluster {k+1}')

    # Plota a movimentação dos centróides
    for j in range(K):
        # Plota a linha de movimentação dos centróides
        plt.plot([previous_centroids[j, 0], centroids[j, 0]], [previous_centroids[j, 1], centroids[j, 1]], 'k--')
        # Plota os centróides atuais
        plt.scatter(centroids[j, 0], centroids[j, 1], s=300, c=colors[j % len(colors)], marker='x', linewidth=2)

    # Define o título do gráfico
    plt.title(f'Iteração {i + 1}')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.legend()
    plt.grid(True)
    plt.show()

def compute_centroids(X, idx, K):
    m, n = X.shape
    centroids = np.zeros((K, n))

    for i in range(K):
        # Para cada centróide i, extraímos os pontos atribuídos a ele com idx == i
        points_assigned_to_centroid = X[idx == i]
        centroids[i] = (1/len(points_assigned_to_centroid)) * np.sum(points_assigned_to_centroid, axis=0)
    
    return centroids

In [ ]:
def kMeans_init_centroids(X, K):
    # Embaralha aleatoriamente os índices dos exemplos em X
    randidx = np.random.permutation(X.shape[0])
    
    # Seleciona os primeiros K exemplos com base na permutação aleatória dos índices
    centroids = X[randidx[:K]]
    
    # Retorna os centróides inicializados aleatoriamente
    return centroids

	
def run_kMeans(X, initial_centroids, max_iters=10, plot_progress=False):
    # Inicializa valores
    m, n = X.shape  # m é o número de exemplos e n é o número de características em X
    K = initial_centroids.shape[0]  # Número de centróides
    centroids = initial_centroids  # Define os centróides iniciais
    previous_centroids = centroids  # Armazena os centróides anteriores
    idx = np.zeros(m)  # Inicializa o array de índices com zeros, do mesmo tamanho que o número de exemplos em X
    plt.figure(figsize=(8, 6))  # Configura o tamanho da figura para a plotagem

    # Executa o K-Means
    for i in range(max_iters):
        # Exibe o progresso
        print("K-Means iteration %d/%d" % (i, max_iters-1))
        
        # Para cada exemplo em X, atribui-o ao centróide mais próximo
        idx = find_closest_centroids(X, centroids)
        
        # Opcionalmente plota o progresso
        if plot_progress:
            plot_progress_kMeans(X, centroids, previous_centroids, idx, K, i)
            previous_centroids = centroids  # Atualiza os centróides anteriores
            
        # Dadas as atribuições, calcula novos centróides
        centroids = compute_centroids(X, idx, K)
    
    plt.show()  # Mostra o gráfico se a plotagem de progresso estiver habilitada
    return centroids, idx  # Retorna os centróides finais e as atribuições de cluster

In [ ]:
# Carregar a imagem
original_img = plt.imread('AI.png')

# Visualizar a imagem
plt.imshow(original_img)

In [ ]:
# Verificando a dimensão da imagem
print("Shape of original_img is:", original_img.shape)

In [ ]:
# Transformar a matriz original_img em uma matriz bidimensional (porque?)
# A matriz resultante terá (número de pixels) linhas e 3 colunas (representando RGB)
X_img = np.reshape(original_img, (original_img.shape[0] * original_img.shape[1], 3))

In [ ]:
# Número de cores (clusters) que queremos encontrar (testar a diferença entre vários K)
K = 16

# Número máximo de iterações para o algoritmo K-Means
max_iters = 10

# Inicializar centróides aleatoriamente a partir dos pixels da imagem
initial_centroids = kMeans_init_centroids(X_img, K)

# Executar o algoritmo K-Means para encontrar os centróides e atribuições de cluster
centroids, idx = run_kMeans(X_img, initial_centroids, max_iters)